# Ionic with LangChain's ChatGoogleGenerativeAI agent and Gemini Pro model

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ioniccommerce/examples/blob/main/examples/notebooks/langchain_react_agent_gemini.ipynb)

### Install dependencies

In [1]:
%%bash

pip install --upgrade pip
pip install langchain langchain-core langchain-google-vertexai langchainhub
pip install ionic-langchain

### Setup Google Auth Key
To use Google's Vertex AI, you will need to [create a Google Cloud project](https://developers.google.com/workspace/guides/create-project), enable billing, and then enable the Vertex AI API.

When you have a project and Vertex enabled, configure your Google Cloud credentials by running the following command in your terminal:

```bash
gcloud auth application-default login
```

This will open a browser window asking you to log in to your Google account and grant permissions to the gcloud command-line tool. After you've done this, you should see a message in your terminal saying that you are now authenticated.

### Create your Agent with the Ionic Commerce Tool

In [2]:
from ionic_langchain.tool import Ionic, IonicTool
from langchain import hub
from langchain.agents import AgentExecutor, Tool, create_react_agent
from langchain_google_vertexai import VertexAI

# Based on ReAct Agent
# https://python.langchain.com/docs/modules/agents/agent_types/react
# Please reach out to support@ionicapi.com for help with add'l agent types.

model = "gemini-pro"
llm = model = VertexAI(model_name=model, )

ionic_tool = IonicTool().tool()

# The tool comes with its own prompt,
# but you may also update it directly via the description attribute:
ionic_tool.description = str(
    """
Ionic is an e-commerce shopping tool. Assistant uses the Ionic Commerce Shopping Tool to find, discover, and compare products from thousands of online retailers. Assistant should use the tool when the user is looking for a product recommendation or trying to find a specific product.

The user may specify the number of results, minimum price, and maximum price for which they want to see results.
Ionic Tool input is a comma-separated string of values:
  - query string (required, must not include commas)
  - number of results (default to 4, no more than 10)
  - minimum price in cents ($5 becomes 500)
  - maximum price in cents
For example, if looking for coffee beans between 5 and 10 dollars, the tool input would be `coffee beans, 5, 500, 1000`.

Return them as a markdown formatted list with each recommendation from tool results, being sure to include the full PDP URL. For example:

1. Product 1: [Price] -- link
2. Product 2: [Price] -- link
3. Product 3: [Price] -- link
4. Product 4: [Price] -- link
"""
)

tools = [ionic_tool]

# default prompt for create_react_agent
prompt = hub.pull("hwchase17/react")

agent = create_react_agent(
    llm,
    tools,
    prompt=prompt,
)

agent_executor = AgentExecutor(
    agent=agent, tools=tools, handle_parsing_errors=True, verbose=True, max_iterations=5
)

### Run the Agent

In [3]:
input = (
    "I'm looking for a 4K Monitor"
)
agent_executor.invoke({"input": input})



> Entering new AgentExecutor chain...
I can use the Ionic Commerce Shopping Tool to find products related to this query.
Action: ionic_commerce_shopping_tool
Action Input: 4K Monitor, 4, 10000, 60000[{'products': [{'links': [{'text': 'Details', 'type': 'pdp', 'url': 'https://brwi.short.gy/ldDEuQ'}], 'merchant_name': 'Walmart', 'merchant_product_id': '5085494806', 'name': 'KOORUI 27 inch IPS Computer Monitor,4K UHD,HDR10,90% DCI-P3,Adpitive-sync,Dp1.4&HDMI2.0 Ports,N07', 'price': '$189.88', 'status': 'available', 'thumbnail': 'https://i5.walmartimages.com/asr/a79af0d8-6338-41c2-99fc-a7a5f7c7933e.70b004740dda930dd21f79415869250f.jpeg?odnHeight=100&odnWidth=100&odnBg=ffffff', 'brand_name': 'KOORUI', 'upc': '679262965879'}, {'links': [{'text': 'Details', 'type': 'pdp', 'url': 'https://www.amazon.com/dp/B098HQ8YRX?tag=ioniccommer00-20&linkCode=osi&th=1&psc=1'}], 'merchant_name': 'Amazon', 'merchant_product_id': 'B098HQ8YRX', 'name': 'Sceptre 4K IPS 27" 3840 x 2160 UHD Monitor up to 70Hz D

{'input': "I'm looking for a 4K Monitor",
 'output': '1. KOORUI 27 inch IPS Computer Monitor: $189.88 -- https://brwi.short.gy/ldDEuQ \n2. Sceptre 4K IPS 27" 3840 x 2160 UHD Monitor: $169.97 -- https://www.amazon.com/dp/B098HQ8YRX?tag=ioniccommer00-20&linkCode=osi&th=1&psc=1\n3. PowerExpand USB-C to DisplayPort Adapter: $19.99 -- https://brwi.short.gy/HFpqjN\n4. SAMSUNG UJ59 Series 32-Inch 4K UHD (3840x2160) Computer Monitor: $279.99 -- https://www.amazon.com/dp/B07CS3JGPC?tag=ioniccommer00-20&linkCode=osi&th=1&psc=1'}